In [ ]:
import getpass
import os
os.environ["http_proxy"] = "http://localhost:7890"
os.environ["https_proxy"] = "http://localhost:7890"
api_key = "sk-zk224a777126590fa5988d720e1c413bd6866854f68d0768"

from langchain_openai import ChatOpenAI
BASE_URL = "https://api.zhizengzeng.com/v1"
llm = ChatOpenAI(api_key=api_key,model="gpt-3.5-turbo",base_url=BASE_URL)


In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field


# Note that the docstrings here are crucial, as they will be passed along
# to the model along with the class name.
class Add(BaseModel):
    """Add two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class Multiply(BaseModel):
    """Multiply two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


tools = [Add, Multiply]

In [ ]:
llm_with_tools = llm.bind_tools(tools)

query = "What is 3 * 12?"

llm_with_tools.invoke(query)

In [ ]:
query = "What is 3 * 12? Also, what is 11 + 49?"

llm_with_tools.invoke(query).tool_calls

In [ ]:
chunks = []
for chunk in llm.stream("what color is the sky?"):
    chunks.append(chunk)
    print(chunk.content, end="|", flush=True)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
import os
os.environ["http_proxy"] = "http://localhost:7890"
os.environ["https_proxy"] = "http://localhost:7890"
api_key = "sk-zk224a777126590fa5988d720e1c413bd6866854f68d0768"

from langchain_openai import ChatOpenAI
BASE_URL = "https://api.zhizengzeng.com/v1"
model = ChatOpenAI(api_key=api_key,model="gpt-3.5-turbo",base_url=BASE_URL)


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Write out the following equation using algebraic symbols then solve it. Use the format\n\nEQUATION:...\nSOLUTION:...\n\n",
        ),
        ("human", "{equation_statement}"),
    ]
)


runnable = (
    {"equation_statement": RunnablePassthrough()} | prompt | model | StrOutputParser()
)

print(runnable.invoke("x raised to the third plus seven equals 12"))

In [ ]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda


from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
import os
os.environ["http_proxy"] = "http://localhost:7890"
os.environ["https_proxy"] = "http://localhost:7890"
api_key = "sk-zk224a777126590fa5988d720e1c413bd6866854f68d0768"

from langchain_openai import ChatOpenAI
BASE_URL = "https://api.zhizengzeng.com/v1"

model = ChatOpenAI(api_key = api_key ,model="gpt-3.5-turbo",base_url=BASE_URL)


def length_function(text):
    return len(text)


def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)


def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])




In [ ]:


prompt = ChatPromptTemplate.from_template("what is {a} + {b}")
    
chain1 = prompt | model

length_function = RunnableLambda(length_function)

In [ ]:
chain = (
    {
        "a":itemgetter("foo") | length_function ,
        "b": {"text1": itemgetter("foo"), "text2": itemgetter("bar")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
)

chain.invoke({"foo": "bar", "bar": "gah"})

### messages history


In [1]:
import os
os.environ["http_proxy"] = "http://localhost:7890"
os.environ["https_proxy"] = "http://localhost:7890"
api_key = os.environ["OPENAI_API_KEY"]

from langchain_openai import ChatOpenAI
BASE_URL = "https://api.zhizengzeng.com/v1"



In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini",base_url=BASE_URL)

In [4]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage

# the simplest example

In [36]:
# define get session history function

from langchain_community.chat_message_histories import SQLChatMessageHistory

def get_session_history(session_id):
    print(SQLChatMessageHistory(session_id, "sqlite:///memory.db"))
    return SQLChatMessageHistory(session_id, "sqlite:///memory.db")

In [37]:
runnable_with_history = RunnableWithMessageHistory(
    llm,
    get_session_history = get_session_history
)

In [38]:
runnable_with_history.invoke(
    [HumanMessage(content="Hi I'm JIN")],
    config={"configurable":{"session_id":"1"}} # like thread
)

Parent run 7dbbf6d4-e40c-4ec4-8ab1-2b7200db713f not found for run 97b2255f-3307-4e13-a022-13530db2bf98. Treating as a root run.


AIMessage(content='Hello, JIN! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 11, 'total_tokens': 23}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_661538dc1f', 'finish_reason': 'stop', 'logprobs': None}, id='run-97b2255f-3307-4e13-a022-13530db2bf98-0', usage_metadata={'input_tokens': 11, 'output_tokens': 12, 'total_tokens': 23})

In [39]:
runnable_with_history.invoke(
    [HumanMessage(content="what's my name")],
    config={"configurable":{"session_id":"1"}}
)

Parent run ec24e921-9549-434b-a0df-fa19cbb1781f not found for run adf341d5-a4a7-4c13-95cf-8672affb1dff. Treating as a root run.


Human: Hi I'm JIN
AI: Hello, JIN! How can I assist you today?


AIMessage(content='Your name is JIN. How can I help you further?', response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 35, 'total_tokens': 48}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_661538dc1f', 'finish_reason': 'stop', 'logprobs': None}, id='run-adf341d5-a4a7-4c13-95cf-8672affb1dff-0', usage_metadata={'input_tokens': 35, 'output_tokens': 13, 'total_tokens': 48})

### runnable  use chain

In [40]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're an assistant who speaks in {language}. Respond in 20 words or fewer",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

runnable = prompt | llm
# we must set input/output key up
runnable_with_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [41]:
runnable_with_history.invoke(
    {"language": "italian", "input": "hi im bob!"},
    config={"configurable": {"session_id": "2"}},
)

Parent run 87f12f12-398c-4402-a5d1-9553dc5a1dc2 not found for run 925e8fcb-8883-4c3a-aec8-f30e2444bfd6. Treating as a root run.


AIMessage(content='Ciao Bob! Come posso aiutarti oggi?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 30, 'total_tokens': 40}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_661538dc1f', 'finish_reason': 'stop', 'logprobs': None}, id='run-350d1bdb-66fd-4cb9-b3f5-b3ce9fec730a-0', usage_metadata={'input_tokens': 30, 'output_tokens': 10, 'total_tokens': 40})

In [42]:
runnable_with_history.invoke(
    {"language": "italian", "input": "whats my name?"},
    config={"configurable": {"session_id": "2"}},
)

Parent run c9c86bc6-7861-49f8-814d-526826574da9 not found for run 64e127c9-4432-459c-9d91-3b8d65f1f8bd. Treating as a root run.


Human: hi im bob!
AI: Ciao Bob! Come posso aiutarti oggi?


AIMessage(content='Il tuo nome è Bob!', response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 53, 'total_tokens': 59}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_661538dc1f', 'finish_reason': 'stop', 'logprobs': None}, id='run-c938fbfc-e14c-4ef3-9869-8e31826def90-0', usage_metadata={'input_tokens': 53, 'output_tokens': 6, 'total_tokens': 59})

In [51]:
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableParallel

chain =RunnableParallel({"output_message":llm})
 

runnable_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    output_messages_key="output_message",
)

In [52]:
runnable_with_history.invoke(
     [HumanMessage(content="hi - im bob!")],
    config={"configurable": {"session_id": "4"}},
)

Parent run 0238ab54-54e1-4edf-a40c-b116338141fc not found for run 9b160302-8f84-4645-ace1-dddd0cacbd14. Treating as a root run.


Human: hi - im bob!
AI: Hi Bob! How can I assist you today?


{'output_message': AIMessage(content='Hi again, Bob! What would you like to chat about today?', response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 35, 'total_tokens': 49}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_661538dc1f', 'finish_reason': 'stop', 'logprobs': None}, id='run-3274e6a8-2925-49c8-bf6c-c32819ac78c1-0', usage_metadata={'input_tokens': 35, 'output_tokens': 14, 'total_tokens': 49})}

In [55]:
from operator import itemgetter

# because the input is a dict , we must use itemgetter to align a key 

runnable_with_history = RunnableWithMessageHistory(
    itemgetter("input_messages") | llm,
    get_session_history,
    input_messages_key="input_messages",
)

In [56]:

runnable_with_history.invoke(
    {"input_messages": [HumanMessage(content="hi - im bob!")]},
    config={"configurable": {"session_id": "5"}},
)

Parent run 2168548c-fdfe-469d-a11d-9df0bc7de7aa not found for run 98bc5d1a-919f-4fe9-a9c3-1b70ff243a32. Treating as a root run.


AIMessage(content='Hi Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 12, 'total_tokens': 22}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_661538dc1f', 'finish_reason': 'stop', 'logprobs': None}, id='run-bd58b2a1-d211-4b1a-bf2b-f87fb0c07a72-0', usage_metadata={'input_tokens': 12, 'output_tokens': 10, 'total_tokens': 22})